## Interpolating from hybrid to pressure levels

In [1]:
import earthkit.meteo.vertical.array as vertical

#### Getting the data

First, we get some sample data containing 137 IFS (full) model levels for two points. This data is defined on full model levels in ascending model level number order. So the first level is model level 1 (i.e. the top), while the last one is model level 137 (i.e. the bottom).

In [2]:
from earthkit.meteo.utils.sample import get_sample

DATA = get_sample("vertical_hybrid_data")
sp = DATA.p_surf # surface pressure [Pa]
t = DATA.t # temperature [K]
q = DATA.q # specific humidity [kg/kg]

In [3]:
sp

array([ 95178.337944  , 102659.81019512])

In [4]:
t.shape, t[::40]

((137, 2),
 array([[197.06175232, 198.54808044],
        [230.29292297, 219.00386047],
        [234.56352234, 229.85160828],
        [264.58778381, 292.04481506]]))

In [5]:
A, B = vertical.hybrid_level_parameters(137, model="ifs")
A.shape, B.shape

((138,), (138,))

#### Using interpolate_hybrid_to_pressure_levels

In [6]:
target_p = [85000., 50000.] # Pa

t_p = vertical.interpolate_hybrid_to_pressure_levels(
    t, target_p, A, B, sp)

for vp, tp in zip(target_p, t_p):
    print(vp, tp)

85000.0 [263.50982742 287.70299692]
50000.0 [238.0038375  259.50822693]


The default interpolation is "linear"; we can change it via the ``interpolation`` kwarg.

In [7]:
t_p = vertical.interpolate_hybrid_to_pressure_levels(
    t, target_p, A, B, sp, interpolation="log")

for vp, tp in zip(target_p, t_p):
    print(vp, tp)

85000.0 [263.51026502 287.70474843]
50000.0 [238.0097753  259.51839286]


##### Using a subset of levels

It is possible to use only a **subset of the model levels** in the input data. This can significantly speed up the computations and reduce memory usage.

To do so the model level range in ``t`` must be contiguous and include the bottom-most level. In the cell below we use only the lowest 50 model levels above the surface. Please note that even though we sliced ``t`` we still need to use the full A and B arrays.

In [8]:
t_p = vertical.interpolate_hybrid_to_pressure_levels(
    t[-50:], target_p, A, B, sp, interpolation="linear")

for vp, tp in zip(target_p, t_p):
    print(vp, tp)

85000.0 [263.50982742 287.70299692]
50000.0 [238.0038375  259.50822693]


When the the target pressure is outside the input pressure range the interpolation results is nan values.

In [9]:
target_p1 = [85000., 25000.] # Pa
t_p = vertical.interpolate_hybrid_to_pressure_levels(
    t[-50:], target_p1, A, B, sp, interpolation="linear")

for vp, tp in zip(target_p1, t_p):
    print(vp, tp)

85000.0 [263.50982742 287.70299692]
25000.0 [nan nan]


#### Using interpolate_monotonic

In [10]:
p = vertical.pressure_on_hybrid_levels(A, B, sp, output="full")
p.shape

(137, 2)

In [11]:
t_p = vertical.interpolate_monotonic(t, p, target_p, interpolation="linear")
for vp, tp in zip(target_p, t_p):
    print(vp, tp)

85000.0 [263.50982742 287.70299692]
50000.0 [238.0038375  259.50822693]


The advantage of this method is that we only need to compute the pressure once and can reuse it for multiple computations. E.g. this is how we interpolate specific humidity.

In [12]:
q_p = vertical.interpolate_monotonic(q, p, target_p, interpolation="linear")
for vp, qp in zip(target_p, q_p):
    print(vp, qp)

85000.0 [0.00119029 0.00621813]
50000.0 [0.00010948 0.00046553]


##### Using a subset of levels

We can also use a **subset of levels**. The code below only use the lowest 50 model levels above the surface.

In [13]:
levels = list(range(138-50, 138)) # 97 - 137

p = vertical.pressure_on_hybrid_levels(A, B, sp, output="full", levels=levels)

t_p = vertical.interpolate_monotonic(t[-50:], p, target_p, interpolation="linear")
for vp, tp in zip(target_p, t_p):
    print(vp, tp)

85000.0 [263.50982742 287.70299692]
50000.0 [238.0038375  259.50822693]
